# 📘 Word 文档 → 文档代码化写作 Demo（增强版）
本 Notebook 演示如何将 Word 文档内容结构化，并用代码自动生成文档，并支持 **Pandoc 转 PDF**。

In [ ]:
# 安装依赖（首次运行时需要）
!pip install pyyaml jinja2
!apt-get update && apt-get install -y pandoc texlive-xetex

## 第一步：准备数据 (products.yaml)
在传统 Word 文档里，你可能直接写：
```
N706B 模块 AT 指令手册
波特率：115200
命令：AT+CSQ → 查询信号强度
```
现在我们把它转成结构化数据：

In [ ]:
import yaml

yaml_content = """
products:
  - name: N706B
    version: V1.0
    baud_rate: 115200
    commands:
      - cmd: AT+CSQ
        desc: 查询信号强度
        example: "+CSQ: 25,0"
  - name: N58-GL
    version: V2.0
    baud_rate: 921600
    commands:
      - cmd: AT+CGATT
        desc: 网络附着
        example: "+CGATT: 1"
"""

with open("products.yaml", "w", encoding="utf-8") as f:
    f.write(yaml_content)

data = yaml.safe_load(yaml_content)
data

## 第二步：准备模板
用 Jinja2 模板定义文档的结构，而不是直接在 Word 里排版。

In [ ]:
from jinja2 import Template

template_str = """
{{ name }} 模块 AT 指令手册 (版本 {{ version }})
========================================

基础信息
--------
* 型号: {{ name }}
* 波特率: {{ baud_rate }}

支持命令
--------
| 指令   | 功能描述       | 示例响应 |
|--------|----------------|----------|
{% for c in commands %}| {{ c.cmd }} | {{ c.desc }} | {{ c.example }} |
{% endfor %}
"""

template = Template(template_str)

## 第三步：生成 Markdown 文档 + PDF
循环读取 YAML 数据，用模板渲染输出 Markdown 文本，并直接用 Pandoc 转 PDF。

In [ ]:
import subprocess

for product in data["products"]:
    md_output = template.render(**product)
    md_file = f"{product['name']}_manual.md"
    pdf_file = f"{product['name']}_manual.pdf"
    
    # 保存 Markdown 文件
    with open(md_file, "w", encoding="utf-8") as f:
        f.write(md_output)
    print(f"生成 Markdown: {md_file}")

    # 调用 Pandoc 转 PDF
    try:
        subprocess.run(["pandoc", md_file, "-o", pdf_file, "--pdf-engine=xelatex"], check=True)
        print(f"✅ 已生成 PDF: {pdf_file}")
    except Exception as e:
        print(f"❌ 生成 PDF 失败: {e}")
    
    print("-"*40)

## 第四步：下一步拓展
- 现在可以一键从 YAML → Markdown → PDF。
- 你可以把 `products.yaml` 替换为任意产品参数，快速生成标准化手册。
- 下一步可以扩展到 Sphinx / DITA-OT，实现大规模文档编译。